## Challenge 9
### PKCS#7 padding

Pretty simple, do as they say challenge

In [3]:
def pkcs_7_pad(unpadded_block, final_len):
    '''
    Assumes unpadded block is ascii
    '''
    pad = final_len - len(unpadded_block)
    return unpadded_block + chr(pad) * pad

print(pkcs_7_pad("YELLOW SUBMARINE", 20) == "YELLOW SUBMARINE\x04\x04\x04\x04")

True


Easy come, easy go
***

## Challenge 10
### CBC mode

Copy the old ECB code and make it encrypt/decrypt with CBC

Oh, the agony. 

In [46]:
# all functions are written to work with bit vectors, i.e. strings of '1', '0'
# some of these things can be generated, but I prefer the hardcoded tables
from IPython.core.debugger import Tracer
from __future__ import division

rjin_s_box = [
["63","7c","77","7b","f2","6b","6f","c5","30","01","67","2b","fe","d7","ab","76"],
["ca","82","c9","7d","fa","59","47","f0","ad","d4","a2","af","9c","a4","72","c0"],
["b7","fd","93","26","36","3f","f7","cc","34","a5","e5","f1","71","d8","31","15"],
["04","c7","23","c3","18","96","05","9a","07","12","80","e2","eb","27","b2","75"],
["09","83","2c","1a","1b","6e","5a","a0","52","3b","d6","b3","29","e3","2f","84"],
["53","d1","00","ed","20","fc","b1","5b","6a","cb","be","39","4a","4c","58","cf"],
["d0","ef","aa","fb","43","4d","33","85","45","f9","02","7f","50","3c","9f","a8"],
["51","a3","40","8f","92","9d","38","f5","bc","b6","da","21","10","ff","f3","d2"],
["cd","0c","13","ec","5f","97","44","17","c4","a7","7e","3d","64","5d","19","73"],
["60","81","4f","dc","22","2a","90","88","46","ee","b8","14","de","5e","0b","db"],
["e0","32","3a","0a","49","06","24","5c","c2","d3","ac","62","91","95","e4","79"],
["e7","c8","37","6d","8d","d5","4e","a9","6c","56","f4","ea","65","7a","ae","08"],
["ba","78","25","2e","1c","a6","b4","c6","e8","dd","74","1f","4b","bd","8b","8a"],
["70","3e","b5","66","48","03","f6","0e","61","35","57","b9","86","c1","1d","9e"],
["e1","f8","98","11","69","d9","8e","94","9b","1e","87","e9","ce","55","28","df"],
["8c","a1","89","0d","bf","e6","42","68","41","99","2d","0f","b0","54","bb","16"]
]


rjin_s_box_inv = [
["52","09","6a","d5","30","36","a5","38","bf","40","a3","9e","81","f3","d7","fb"],
["7c","e3","39","82","9b","2f","ff","87","34","8e","43","44","c4","de","e9","cb"],
["54","7b","94","32","a6","c2","23","3d","ee","4c","95","0b","42","fa","c3","4e"],
["08","2e","a1","66","28","d9","24","b2","76","5b","a2","49","6d","8b","d1","25"],
["72","f8","f6","64","86","68","98","16","d4","a4","5c","cc","5d","65","b6","92"],
["6c","70","48","50","fd","ed","b9","da","5e","15","46","57","a7","8d","9d","84"],
["90","d8","ab","00","8c","bc","d3","0a","f7","e4","58","05","b8","b3","45","06"],
["d0","2c","1e","8f","ca","3f","0f","02","c1","af","bd","03","01","13","8a","6b"],
["3a","91","11","41","4f","67","dc","ea","97","f2","cf","ce","f0","b4","e6","73"],
["96","ac","74","22","e7","ad","35","85","e2","f9","37","e8","1c","75","df","6e"],
["47","f1","1a","71","1d","29","c5","89","6f","b7","62","0e","aa","18","be","1b"],
["fc","56","3e","4b","c6","d2","79","20","9a","db","c0","fe","78","cd","5a","f4"],
["1f","dd","a8","33","88","07","c7","31","b1","12","10","59","27","80","ec","5f"],
["60","51","7f","a9","19","b5","4a","0d","2d","e5","7a","9f","93","c9","9c","ef"],
["a0","e0","3b","4d","ae","2a","f5","b0","c8","eb","bb","3c","83","53","99","61"],
["17","2b","04","7e","ba","77","d6","26","e1","69","14","63","55","21","0c","7d"]
] # usage -> row -> first nibble, col -> second nibble

######## mix col
inv_mix_col_mat = [
    ['0e', '0b', '0d', '09'],
    ['09', '0e', '0b', '0d'],
    ['0d', '09', '0e', '0b'],
    ['0b', '0d', '09', '0e']
]
mix_col_mat = [
    ['02', '03', '01', '01'],
    ['01', '02', '03', '01'],
    ['01', '01', '02', '03'],
    ['03', '01', '01', '02']    
]

g_mul = {}
g_mul[14] = [
"00","0e","1c","12","38","36","24","2a","70","7e","6c","62","48","46","54","5a",
"e0","ee","fc","f2","d8","d6","c4","ca","90","9e","8c","82","a8","a6","b4","ba",
"db","d5","c7","c9","e3","ed","ff","f1","ab","a5","b7","b9","93","9d","8f","81",
"3b","35","27","29","03","0d","1f","11","4b","45","57","59","73","7d","6f","61",
"ad","a3","b1","bf","95","9b","89","87","dd","d3","c1","cf","e5","eb","f9","f7",
"4d","43","51","5f","75","7b","69","67","3d","33","21","2f","05","0b","19","17",
"76","78","6a","64","4e","40","52","5c","06","08","1a","14","3e","30","22","2c",
"96","98","8a","84","ae","a0","b2","bc","e6","e8","fa","f4","de","d0","c2","cc",
"41","4f","5d","53","79","77","65","6b","31","3f","2d","23","09","07","15","1b",
"a1","af","bd","b3","99","97","85","8b","d1","df","cd","c3","e9","e7","f5","fb",
"9a","94","86","88","a2","ac","be","b0","ea","e4","f6","f8","d2","dc","ce","c0",
"7a","74","66","68","42","4c","5e","50","0a","04","16","18","32","3c","2e","20",
"ec","e2","f0","fe","d4","da","c8","c6","9c","92","80","8e","a4","aa","b8","b6",
"0c","02","10","1e","34","3a","28","26","7c","72","60","6e","44","4a","58","56",
"37","39","2b","25","0f","01","13","1d","47","49","5b","55","7f","71","63","6d",
"d7","d9","cb","c5","ef","e1","f3","fd","a7","a9","bb","b5","9f","91","83","8d"]

g_mul[11] = [
"00","0b","16","1d","2c","27","3a","31","58","53","4e","45","74","7f","62","69",
"b0","bb","a6","ad","9c","97","8a","81","e8","e3","fe","f5","c4","cf","d2","d9",
"7b","70","6d","66","57","5c","41","4a","23","28","35","3e","0f","04","19","12",
"cb","c0","dd","d6","e7","ec","f1","fa","93","98","85","8e","bf","b4","a9","a2",
"f6","fd","e0","eb","da","d1","cc","c7","ae","a5","b8","b3","82","89","94","9f",
"46","4d","50","5b","6a","61","7c","77","1e","15","08","03","32","39","24","2f",
"8d","86","9b","90","a1","aa","b7","bc","d5","de","c3","c8","f9","f2","ef","e4",
"3d","36","2b","20","11","1a","07","0c","65","6e","73","78","49","42","5f","54",
"f7","fc","e1","ea","db","d0","cd","c6","af","a4","b9","b2","83","88","95","9e",
"47","4c","51","5a","6b","60","7d","76","1f","14","09","02","33","38","25","2e",
"8c","87","9a","91","a0","ab","b6","bd","d4","df","c2","c9","f8","f3","ee","e5",
"3c","37","2a","21","10","1b","06","0d","64","6f","72","79","48","43","5e","55",
"01","0a","17","1c","2d","26","3b","30","59","52","4f","44","75","7e","63","68",
"b1","ba","a7","ac","9d","96","8b","80","e9","e2","ff","f4","c5","ce","d3","d8",
"7a","71","6c","67","56","5d","40","4b","22","29","34","3f","0e","05","18","13",
"ca","c1","dc","d7","e6","ed","f0","fb","92","99","84","8f","be","b5","a8","a3"
]

g_mul[13] = [
"00","0d","1a","17","34","39","2e","23","68","65","72","7f","5c","51","46","4b",
"d0","dd","ca","c7","e4","e9","fe","f3","b8","b5","a2","af","8c","81","96","9b",
"bb","b6","a1","ac","8f","82","95","98","d3","de","c9","c4","e7","ea","fd","f0",
"6b","66","71","7c","5f","52","45","48","03","0e","19","14","37","3a","2d","20",
"6d","60","77","7a","59","54","43","4e","05","08","1f","12","31","3c","2b","26",
"bd","b0","a7","aa","89","84","93","9e","d5","d8","cf","c2","e1","ec","fb","f6",
"d6","db","cc","c1","e2","ef","f8","f5","be","b3","a4","a9","8a","87","90","9d",
"06","0b","1c","11","32","3f","28","25","6e","63","74","79","5a","57","40","4d",
"da","d7","c0","cd","ee","e3","f4","f9","b2","bf","a8","a5","86","8b","9c","91",
"0a","07","10","1d","3e","33","24","29","62","6f","78","75","56","5b","4c","41",
"61","6c","7b","76","55","58","4f","42","09","04","13","1e","3d","30","27","2a",
"b1","bc","ab","a6","85","88","9f","92","d9","d4","c3","ce","ed","e0","f7","fa",
"b7","ba","ad","a0","83","8e","99","94","df","d2","c5","c8","eb","e6","f1","fc",
"67","6a","7d","70","53","5e","49","44","0f","02","15","18","3b","36","21","2c",
"0c","01","16","1b","38","35","22","2f","64","69","7e","73","50","5d","4a","47",
"dc","d1","c6","cb","e8","e5","f2","ff","b4","b9","ae","a3","80","8d","9a","97"
]

g_mul[9] = [
"00","09","12","1b","24","2d","36","3f","48","41","5a","53","6c","65","7e","77",
"90","99","82","8b","b4","bd","a6","af","d8","d1","ca","c3","fc","f5","ee","e7",
"3b","32","29","20","1f","16","0d","04","73","7a","61","68","57","5e","45","4c",
"ab","a2","b9","b0","8f","86","9d","94","e3","ea","f1","f8","c7","ce","d5","dc",
"76","7f","64","6d","52","5b","40","49","3e","37","2c","25","1a","13","08","01",
"e6","ef","f4","fd","c2","cb","d0","d9","ae","a7","bc","b5","8a","83","98","91",
"4d","44","5f","56","69","60","7b","72","05","0c","17","1e","21","28","33","3a",
"dd","d4","cf","c6","f9","f0","eb","e2","95","9c","87","8e","b1","b8","a3","aa",
"ec","e5","fe","f7","c8","c1","da","d3","a4","ad","b6","bf","80","89","92","9b",
"7c","75","6e","67","58","51","4a","43","34","3d","26","2f","10","19","02","0b",
"d7","de","c5","cc","f3","fa","e1","e8","9f","96","8d","84","bb","b2","a9","a0",
"47","4e","55","5c","63","6a","71","78","0f","06","1d","14","2b","22","39","30",
"9a","93","88","81","be","b7","ac","a5","d2","db","c0","c9","f6","ff","e4","ed",
"0a","03","18","11","2e","27","3c","35","42","4b","50","59","66","6f","74","7d",
"a1","a8","b3","ba","85","8c","97","9e","e9","e0","fb","f2","cd","c4","df","d6",
"31","38","23","2a","15","1c","07","0e","79","70","6b","62","5d","54","4f","46"
]

g_mul[2] = [
"00","02","04","06","08","0a","0c","0e","10","12","14","16","18","1a","1c","1e",
"20","22","24","26","28","2a","2c","2e","30","32","34","36","38","3a","3c","3e",
"40","42","44","46","48","4a","4c","4e","50","52","54","56","58","5a","5c","5e",
"60","62","64","66","68","6a","6c","6e","70","72","74","76","78","7a","7c","7e",
"80","82","84","86","88","8a","8c","8e","90","92","94","96","98","9a","9c","9e",
"a0","a2","a4","a6","a8","aa","ac","ae","b0","b2","b4","b6","b8","ba","bc","be",
"c0","c2","c4","c6","c8","ca","cc","ce","d0","d2","d4","d6","d8","da","dc","de",
"e0","e2","e4","e6","e8","ea","ec","ee","f0","f2","f4","f6","f8","fa","fc","fe",
"1b","19","1f","1d","13","11","17","15","0b","09","0f","0d","03","01","07","05",
"3b","39","3f","3d","33","31","37","35","2b","29","2f","2d","23","21","27","25",
"5b","59","5f","5d","53","51","57","55","4b","49","4f","4d","43","41","47","45",
"7b","79","7f","7d","73","71","77","75","6b","69","6f","6d","63","61","67","65",
"9b","99","9f","9d","93","91","97","95","8b","89","8f","8d","83","81","87","85",
"bb","b9","bf","bd","b3","b1","b7","b5","ab","a9","af","ad","a3","a1","a7","a5",
"db","d9","df","dd","d3","d1","d7","d5","cb","c9","cf","cd","c3","c1","c7","c5",
"fb","f9","ff","fd","f3","f1","f7","f5","eb","e9","ef","ed","e3","e1","e7","e5"
]

g_mul[3] = [
"00","03","06","05","0c","0f","0a","09","18","1b","1e","1d","14","17","12","11",
"30","33","36","35","3c","3f","3a","39","28","2b","2e","2d","24","27","22","21",
"60","63","66","65","6c","6f","6a","69","78","7b","7e","7d","74","77","72","71",
"50","53","56","55","5c","5f","5a","59","48","4b","4e","4d","44","47","42","41",
"c0","c3","c6","c5","cc","cf","ca","c9","d8","db","de","dd","d4","d7","d2","d1",
"f0","f3","f6","f5","fc","ff","fa","f9","e8","eb","ee","ed","e4","e7","e2","e1",
"a0","a3","a6","a5","ac","af","aa","a9","b8","bb","be","bd","b4","b7","b2","b1",
"90","93","96","95","9c","9f","9a","99","88","8b","8e","8d","84","87","82","81",
"9b","98","9d","9e","97","94","91","92","83","80","85","86","8f","8c","89","8a",
"ab","a8","ad","ae","a7","a4","a1","a2","b3","b0","b5","b6","bf","bc","b9","ba",
"fb","f8","fd","fe","f7","f4","f1","f2","e3","e0","e5","e6","ef","ec","e9","ea",
"cb","c8","cd","ce","c7","c4","c1","c2","d3","d0","d5","d6","df","dc","d9","da",
"5b","58","5d","5e","57","54","51","52","43","40","45","46","4f","4c","49","4a",
"6b","68","6d","6e","67","64","61","62","73","70","75","76","7f","7c","79","7a",
"3b","38","3d","3e","37","34","31","32","23","20","25","26","2f","2c","29","2a",
"0b","08","0d","0e","07","04","01","02","13","10","15","16","1f","1c","19","1a"
]


def poly_field_multiply(mix_col_mat_val, v2_bin):
    v1 = int(mix_col_mat_val, 16)
    v2 = int(v2_bin, 2)
    if v1 != 1:
        mul = int(g_mul[v1][v2], 16)
    else:
        mul = v2
    return bin(mul)[2:].zfill(8)

#######
# someday we will write documentation for each of these functions, but today is not that day

def inv_sub_byte_single(byte):
    row = int(byte[:4], 2)
    col = int(byte[4:], 2)
    val = int(rjin_s_box_inv[row][col], 16)
    return bin(val)[2:].zfill(8)


def shift_right(row):
    rowcopy = list(row)
    last = rowcopy[-1]
    for i in range(len(row) - 1):
        row[i+1] = rowcopy[i]
    row[0] = last


def shift_left(row):
    rowcopy = list(row)
    first = rowcopy[0]
    for i in range(1, len(row)):
        row[i-1] = rowcopy[i]
    row[-1] = first

def bit_str_xor(s1, s2):
    if len(s1) != len(s2):
        print(s1, s2, "problem")
        Tracer()()
    sout = ""
    for i in range(len(s1)):
        if s1[i] == s2[i]:
            sout += '0'
        else:
            sout += '1'
    
    return sout


def sub_byte_single(byte):
    row = int(byte[:4], 2)
    col = int(byte[4:], 2)
    val = int(rjin_s_box[row][col], 16)
    return bin(val)[2:].zfill(8)

def inv_sub_bytes(state_mat):
    for i in range(4):
        for j in range(4):
            state_mat[i][j] = inv_sub_byte_single(state_mat[i][j])

def sub_bytes(state_mat):
    for i in range(4):
            for j in range(4):
                state_mat[i][j] = sub_byte_single(state_mat[i][j])


def shift_rows(state_mat):
    for i in range(4):
        row = state_mat[i]
        for j in range(i):
            shift_left(row)


def inv_shift_rows(state_mat):
    for i in range(4):
        row = state_mat[i]
        for j in range(i):
            shift_right(row)

            
def inv_mix_cols(state_mat):
    '''
    This one actually returns a value. The rest of the functions change state
    '''
    fin_mat = [list(row) for row in state_mat]
    
    for i in range(4):
        for j in range(4):
            accum = poly_field_multiply(inv_mix_col_mat[i][0], state_mat[0][j])
            for k in range(1, 4):
                prod = poly_field_multiply(inv_mix_col_mat[i][k], state_mat[k][j])
                accum = bit_str_xor(accum, prod)
            
            fin_mat[i][j] = accum
    
    return fin_mat

def mix_cols(state_mat):
    '''
    This one actually returns a value. 
    '''
    fin_mat = [list(row) for row in state_mat]
    
    for i in range(4):
        for j in range(4):
            accum = poly_field_multiply(mix_col_mat[i][0], state_mat[0][j])
            for k in range(1, 4):
                prod = poly_field_multiply(mix_col_mat[i][k], state_mat[k][j])
                accum = bit_str_xor(accum, prod)
            
            fin_mat[i][j] = accum
    
    return fin_mat


def add_round_key(state_mat, round_key):
    for i in range(4):
        for j in range(4):
            state_mat[i][j] = bit_str_xor(state_mat[i][j], round_key[i][j])

def create_key_expand(key_init):
    '''
    Takes a 16 byte key and returns a 4 x 44 byte array with each column corresponding to a key word
    '''
    expanded_key = [['00000000' for j in range(44)]  for i in range(4)]
    
    #init first 4 cols
    for i in range(16):
        byte = key_init[8*i:8*(i+1)]
        row = i%4
        col = i//4
        expanded_key[row][col] = byte
    
    RC = ['00000000', '00000001', '00000010', '00000100', '00001000', '00010000', '00100000', '01000000', '10000000', '00011011', '00110110', '01101100']
    
    for rno in range(1, 11):
        col = 4 * rno
        
        # Tracer()()
        # first get w_col
        prev_word = [""] * 4 
        
        for j in range(4):
            prev_word[j] = expanded_key[j][col-1]
        
        shift_left(prev_word)
        
#         for j in range(4):
#             expanded_key[j][col-1] = prev_word[j]

        for j in range(4):
            prev_word[j] = sub_byte_single(prev_word[j])
        
        prev_word[0] = bit_str_xor(prev_word[0], RC[rno])
        
        g_col = list(prev_word)
        
        w_col = [""] * 4
        for j in range(4):
            w_col[j] = bit_str_xor(expanded_key[j][col-4], g_col[j])
        
        #now w_col+1, w_col+2, w_col+3
        w_col1 = [""] * 4
        w_col2 = [""] * 4
        w_col3 = [""] * 4
        for j in range(4):
            w_col1[j] = bit_str_xor(expanded_key[j][col-3], w_col[j])
            w_col2[j] = bit_str_xor(expanded_key[j][col-2], w_col1[j])
            w_col3[j] = bit_str_xor(expanded_key[j][col-1], w_col2[j])
        
        #now fill the values into expanded key
        for j in range(4):
            expanded_key[j][col] = w_col[j]
            expanded_key[j][col+1] = w_col1[j]
            expanded_key[j][col+2] = w_col2[j]
            expanded_key[j][col+3] = w_col3[j]
    
    return expanded_key
    

def aes_decrypt_block_ecb(ciphertext, key):
    '''Assumes both ciphertext and key are bit strings, works for 16 byte ciphertext'''
    #initialise the key and state mat
    expanded_key = create_key_expand(key)
    state_mat = [["" for x in range(4)] for y in range(4)]
    for i in range(16):
        byte = ciphertext[8*i:8*(i+1)]
        row = i%4
        col = i//4
        state_mat[row][col] = byte

    # apply preliminary key
    st_col = 40
    cur_round_key = [["" for x in range(4)] for y in range(4)]
    for j in range(4):
        for i in range(4):
            cur_round_key[i][j] = expanded_key[i][st_col + j]
    
    add_round_key(state_mat, cur_round_key)
    
    # invert 10 rounds
    for rno in range(1, 11):
        #inverse shift rows
        inv_shift_rows(state_mat)
        
        #inverse substitute bytes
        inv_sub_bytes(state_mat)
        
        #add round key
        st_col -= 4
        ##generate round key
        for j in range(4):
            for i in range(4):
                cur_round_key[i][j] = expanded_key[i][st_col + j]
        
        add_round_key(state_mat, cur_round_key)
        
        #inverse mix columns, except in last round
        if rno != 10:
            state_mat = inv_mix_cols(state_mat)
    
    fin_plaintext = ""
    for j in range(4):
        for i in range(4):
            fin_plaintext += state_mat[i][j]
    
    return fin_plaintext

def aes_encrypt_block_ecb(plaintext, key):
    '''Assumes both plaintext and key are bit strings, works for 16 byte ciphertext'''
    #initialise the key and state mat
    expanded_key = create_key_expand(key)
    state_mat = [["" for x in range(4)] for y in range(4)]
    for i in range(16):
        byte = plaintext[8*i:8*(i+1)]
        row = i%4
        col = i//4
        state_mat[row][col] = byte

    # apply preliminary key
    st_col = 0
    cur_round_key = [["" for x in range(4)] for y in range(4)]
    for j in range(4):
        for i in range(4):
            cur_round_key[i][j] = expanded_key[i][st_col + j]
    
    add_round_key(state_mat, cur_round_key)
    
    # invert 10 rounds
    for rno in range(1, 11):
        #substitute bytes
        sub_bytes(state_mat)
        
        #shift rows
        shift_rows(state_mat)
        
        #mix columns, except in last round
        if rno != 10:
            state_mat = mix_cols(state_mat)

        #add round key
        st_col += 4
        ##generate round key
        for j in range(4):
            for i in range(4):
                cur_round_key[i][j] = expanded_key[i][st_col + j]
        
        add_round_key(state_mat, cur_round_key)

    
    fin_ciphertext = ""
    for j in range(4):
        for i in range(4):
            fin_ciphertext += state_mat[i][j]
    
    return fin_ciphertext


def aes_decrypt_ecb(ciphertext, key):
    plaintext = ""
    for i in range(len(ciphertext)//128):
        plaintext += aes_decrypt_block_ecb(ciphertext[128*i:128*(i+1)], key)
    
    return plaintext

def aes_encrypt_ecb(plaintext, key):
    ciphertext = ""
    for i in range(len(plaintext) // 128):
        ciphertext += aes_encrypt_block_ecb(plaintext[128*i:128*(i+1)], key)
        
    return ciphertext
    
def aes_encrypt_cbc(plaintext, key, iv):
    ciphertext = str(iv)
    for i in range(len(plaintext) // 128):
        prev_ciph = ciphertext[-128:]
        cur_plain = plaintext[i*128:(i+1)*128]
        round_input = bit_str_xor(cur_plain, prev_ciph)
        ciphertext += aes_encrypt_block_ecb(round_input, key)
    
    return ciphertext[128:]
    
    
    
def aes_decrypt_cbc(ciphertext, key, iv):
    ivciphertext = iv + ciphertext
    plaintext = ""
    for i in range(len(ciphertext) // 128):
        block_start = (len(ciphertext) // 128) - 1 - i
        prev_ciph = ivciphertext[128*block_start:128*(block_start+1)]
        ciphertext_block = ciphertext[128*block_start:128*(block_start+1)]
        plaintext_block = bit_str_xor(prev_ciph, aes_decrypt_block_ecb(ciphertext_block, key))
        plaintext = plaintext_block + plaintext
    
    return plaintext

In [48]:
def ascii_to_bin(asciistr):
    binarr = [bin(ord(c))[2:] for c in asciistr]
    binstr = ''.join([x.zfill(8) for x in binarr])
    return binstr

def bin_to_ascii(binstr):
    asciistr = ""
    for i in range(len(binstr)//8):
        byte = binstr[8*i:8*(i+1)]
        asciistr += chr(int(byte, 2))
    
    return asciistr

test_aes_bin_plaintext = ascii_to_bin("~_HELLO_DARKNESS_MY_OLD_FRIEND_~")
test_aes_bin_key = ascii_to_bin("YELLOW SUBMARINE")

test_aes_bin_ciphertext_ecb = aes_encrypt_ecb(test_aes_bin_plaintext, test_aes_bin_key)
print(bin_to_ascii(aes_decrypt_ecb(test_aes_bin_ciphertext_ecb, test_aes_bin_key)))

~_HELLO_DARKNESS_MY_OLD_FRIEND_~


^^^ It works! :) ^^^

In [49]:
iv = "0" * 128
test_aes_bin_ciphertext_cbc = aes_encrypt_cbc(test_aes_bin_plaintext, test_aes_bin_key, iv)
print(bin_to_ascii(aes_decrypt_cbc(test_aes_bin_ciphertext_cbc, test_aes_bin_key, iv)))

~_HELLO_DARKNESS_MY_OLD_FRIEND_~


^^^ This works too :) ^^^ Onward to the actual string then

In [50]:
import string
base64_arr = list(string.ascii_uppercase)
base64_arr.extend(list(string.ascii_lowercase))
base64_arr.extend(map(str, list(range(0,10))))
base64_arr.extend(["+", "/"])

def base64_to_bin(base64str):
    binstr = ""
    for c in base64str:
        intval = base64_arr.index(c)
        binval = bin(intval)[2:].zfill(6)
        binstr += binval
    
    return binstr

with open("10.txt") as f:
    ciphertext_base64_cbc = ''.join(f.read().splitlines())

ciphertext_bin_cbc = base64_to_bin(ciphertext_base64_cbc)
iv = "0" * 128
key_bin_cbc = ascii_to_bin("YELLOW SUBMARINE")

print(bin_to_ascii(aes_decrypt_cbc(ciphertext_bin_cbc, key_bin_cbc, iv)))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


**(Somewhat) intelligible. HAHAHA**
***

## Challenge 11
### ECB/CBC detection oracle

ECB suffers from the issue that the same plaintext returns the same ciphertext. The chances of this happening in CBC are extremely low, so that should be a sufficient check.

The original problem statement:

```
Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:

encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.
```

In [105]:
import random
rand_gen = random.SystemRandom()

def gen_rand_aes_key():
    '''
    Generate random 16 byte aes key, returned as a bit string of 128 chars
    '''
    return bin(rand_gen.getrandbits(128))[2:].zfill(128)

def aes_encrption_oracle(ascii_input):
    '''
    Takes ASCII input and produces ascii, string output
    '''
    random_key = gen_rand_aes_key()
    byte_append = "01010101"
    random_pre_len = rand_gen.choice(range(5,11))
    random_post_len = rand_gen.choice(range(5,11))
    bin_input = (byte_append * random_pre_len) + ascii_to_bin(ascii_input) + (byte_append * random_post_len)
    
    #### no specification for padding given
    pad = "0"
    if len(bin_input) % 128 != 0:
        req_pad_len = 128 - (len(bin_input) % 128)
        bin_input += pad * req_pad_len
    
    # 0 -> ECB, 1 -> CBC
    rand_cipher_choice = rand_gen.choice(range(2))
    if rand_cipher_choice == 0:
        ciphertext = aes_encrypt_ecb(bin_input, random_key)
        mode = "ecb"
    else:
        random_iv = gen_rand_aes_key() # because the same length
        ciphertext = aes_encrypt_cbc(bin_input, random_key, random_iv)
        mode = "cbc"
        
    return (ciphertext, mode) 
    

In [178]:
from collections import Counter

def detect_aes_mode(ciphertext):
    '''
    Assumes ciphertext is bin
    '''
    block_counter = Counter()
    for i in range(len(ciphertext) // 128):
        block = ciphertext[128*i:128*(i+1)]
        block_counter[block] += 1
        #print(block)

    mfe = block_counter.most_common(1)[0][1]
    
    if mfe > 1:
        return "ecb"
    return "cbc"

In [188]:
for i in range(30):
    with open("test_input.txt") as f:
        test_input_1 = f.read().splitlines()
    
    test_input_1 = aes_encrption_oracle(ascii_to_bin(''.join(test_input_1)))
    test_predict_1 = detect_aes_mode(test_input_1[0])
    print(test_input_1[1], test_predict_1)

ecb ecb
cbc cbc
ecb ecb
ecb ecb
ecb ecb
cbc cbc
ecb ecb
cbc cbc
ecb ecb
ecb ecb
ecb ecb
ecb ecb
ecb ecb
cbc cbc
ecb ecb
ecb ecb
cbc cbc
cbc cbc
cbc cbc
cbc cbc
cbc cbc
cbc cbc
ecb ecb
cbc cbc
cbc cbc
ecb ecb
cbc cbc
ecb ecb
cbc cbc
ecb ecb


These results seem good.

***

## Challenge 12
### Byte-at-a-time ECB decryption (Simple)

This sounds like a cool challenge. 

**Instructions:**

* Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
* Detect that the function is using ECB. You already know, but do this step anyways.

In [189]:
global_random_key = gen_rand_aes_key()

In [193]:
def aes_encryption_oracle_ecb(ascii_input):
    '''
    Takes ASCII input and produces ascii, string output
    '''
    magic_string = base64_to_bin("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
    bin_input =  ascii_to_bin(ascii_input) + magic_string
    ciphertext = aes_encrypt_ecb(bin_input, global_random_key)
        
    return ciphertext 

In [210]:
for i in range(20):
    inp = ascii_to_bin("A" * i)
    ciphertext_test = aes_encryption_oracle_ecb(inp)
    print( "Length of input:", i, ", Length of output:", len(ciphertext_test))
    print(bin_to_ascii(ciphertext_test))
    print("***")
    

Length of input: 0 , Length of output: 1024
¨6^	á MèkÙ"+Ïþß*!ä:×ä69n?ÿJMKýL?Q¤ôOÿ¬+ucÂfí½E«5±¡ÏæÈÝÈÎ C(è½nY+bI¿¶M½fiÐ%§ù£þ¤îe©ôm Í)-e½Éu×õAú1ÜtMåÔ
***
Length of input: 1 , Length of output: 1152
±äæ \Ò,ã£e¦ÃÞç5`jèÚ0³ºj½MGrÈ÷ÐóìHóT¬½h7p/câb[S['WÎLã8ØxLDKVWc¢ó]iã'VÂFëÿhS©& ¤½ßÃ`µòRöÊkK¼mFyht$1¡ µ5Ü¼Y&[,­çéÇ
***
Length of input: 2 , Length of output: 1152
QBÂTð:Ò[Ók×¨6^	á MèkÙ"+Ïþß*!ä:×ä69n?ÿJMKýL?Q¤ôOÿ¬+ucÂfí½E«5±¡ÏæÈÝÈÎ C(è½nY+bI¿¶M½fiÐ%§ù£þ¤îe©ôm Í)-e½Éu×õAú1ÜtMåÔ
***
Length of input: 3 , Length of output: 1280
QBÂTð:Ò[Ók×±äæ \Ò,ã£e¦ÃÞç5`jèÚ0³ºj½MGrÈ÷ÐóìHóT¬½h7p/câb[S['WÎLã8ØxLDKVWc¢ó]iã'VÂFëÿhS©& ¤½ßÃ`µòRöÊkK¼mFyht$1¡ µ5Ü¼Y&[,­çéÇ
***
Length of input: 4 , Length of output: 1280
QBÂTð:Ò[Ók×QBÂTð:Ò[Ók×¨6^	á MèkÙ"+Ïþß*!ä:×ä69n?ÿJMKýL?Q¤ôOÿ¬+ucÂfí½E«5±¡ÏæÈÝÈÎ C(è½nY+bI¿¶M½fiÐ%§ù£þ¤îe©ôm Í)-e½Éu×õAú1ÜtMåÔ
***
Length of input: 5 

Output length increases in multiples of 16 => That must be the block size (additional blocks lead to larger output)

In [202]:
with open("test_input.txt") as f:
    test_input = f.read().splitlines()

test_input = aes_encryption_oracle_ecb(ascii_to_bin(''.join(test_input)))
test_predict_1 = detect_aes_mode(test_input)

In [203]:
print(test_predict_1)

ecb


The mode predicted is ECB.

**Instructions contd:**
* Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
* Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
* Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
* Repeat for the next byte.

Ok, if the block size was 8, and I gave AAAAAAA as the input, and I had the oracle response for AAAAAAAA, AAAAAAAAB, ... etc, the dictionary should be sufficient to get the first byte from the message by comparing with the ciphertext.

Now, I update my 7 known bytes to AAAAAAA[insert learnt byte here]. Again, I can guess the next byte.

So the procedure should be:

1. cur-pad = "A"$\times$7
2. Get ciph for cur-pad -> ciph_1
3. Get oracle response for (cur-pad + poss ASCII chars) 
4. Compare ciph_1 to get the matching oracle response
5. update cur-pad last char. Remove first char (so cur_pad is "A" $\times$ 6, but dictionary is built with known char). Repeat.

This will give me one block of plaintext. Now, I know the first block of plaintext, and I want to get the next block.

To know one byte of the second block, I must push all but 1 known byte of the second block into the third and then build the dictionary. It's a bit difficult to express as a clean algorithm now, and I want it to be.


There are three things here:
* Padding used.
* Dictionary being built
* Ciphertext to compare against

Padding is non trivial to start with. 

Dictionary is easy -> Remove the first character of the previous dictionary, add the most recently learned character to the end, and try all ASCII combinations.

Ciphertext to compare against: 
For the first block, it would be the first block of ciphertext: Here the padding would be "A" $\times$ {15, 14,  ..., 1, 0}
For the second block, the second block of ciphertext, with the padding as "A" $\times$ {15, 14, ..., 1, 0}

Ah, now I see the pattern. Building dictionaries will take computation, but the padded ciphertexts needed are just {blocksize}. Since the total message size is 128, we have 8 blocks. 

In [223]:
# first generate all padded ciphertexts, 16 is all we need

padded_ciphertexts_list = []
for i in range(16):
    pad_len = 16 - i - 1
    pad = "A" * pad_len
    ciphertext = aes_encryption_oracle_ecb(pad)
    padded_ciphertexts_list.append(ciphertext)

# now the actual break
cracked_plaintext = ""
cur_str = "A" * 15
for op_block in range(8):
    for byte in range(16):
        output_dict = {}

        for ch in range(128):
            get_op_str = aes_encryption_oracle_ecb(cur_str + chr(ch))[:128]
            output_dict[get_op_str] = chr(ch)

        compare_str = padded_ciphertexts_list[byte][128*op_block:128*(op_block+1)]
        new_chr = output_dict[compare_str]

        cur_str = cur_str[1:] + new_chr
        cracked_plaintext += new_chr
        
        print(new_chr, end="")
        

print("\n***")
print(cracked_plaintext)
print("***")

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just
***
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just
***


Sweet!

I'm not sure which is growing faster - my knowledge of Vanilla Ice songs, or that of crypto.

```
If our experience is any guideline, this attack will get you code execution in security tests about once a year.
```
Ominous 
***

## Challenge 13
### ECB cut-and-paste

One step at a time:

Write a k=v parsing routine, as if for a structured cookie. The routine should take:
```
foo=bar&baz=qux&zap=zazzle
```
... and produce:
```
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
```